In [ ]:
pip install workalendar

In [ ]:
pip install holidays

In [ ]:
pip install vacances-scolaires-france

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta, date
import calendar
from vacances_scolaires_france import SchoolHolidayDates
import holidays

In [ ]:
data = pd.read_csv('initial_dataset.csv')
data.info()

In [ ]:
data['equipement'][data['equipement'] == 'téléphone'] = 'telephone'

In [ ]:
items_to_retrieve = ['ordinateur', 'telephone']
cities_to_retrieve = ['Bordeaux', 'Mont de Marsan']
data_cities = data[data['ville'].isin(cities_to_retrieve) & (data['equipement'].isin(items_to_retrieve))]
data_cities.info()

In [ ]:
CA_cum = data_cities.groupby(by=['ville', 'equipement']).sum().reset_index()
CA_cum

In [ ]:
data_cities.loc[:, 'processed_date'] = data_cities.loc[:, 'date'].apply(lambda row : datetime(int(row.split('-', 2)[0]), int(row.split('-', 2)[1]), int(row.split('-', 2)[2])))
data_cities['previous_year'] = data_cities['processed_date'].apply(lambda row : row - timedelta(days=365))
data_cities['previous_year_same_day'] = data_cities['processed_date'].apply(lambda row : row - timedelta(days=364))
data_cities.loc[:, 'processed_previous_year'] = data_cities.loc[:, 'previous_year'].apply(lambda row : str(row.year) + '-' + str(row.month).zfill(2) + '-' + str(row.day).zfill(2))
data_cities.loc[:, 'processed_previous_year_same_day'] = data_cities.loc[:, 'previous_year_same_day'].apply(lambda row : str(row.year) + '-' + str(row.month).zfill(2) + '-' + str(row.day).zfill(2))

In [ ]:
CA_previous_year = []

for k in range(len(data_cities)) :
    date_test = data_cities['date'] == data_cities.iloc[k, :]['processed_previous_year']
    ville_test = data_cities['ville'] == data_cities.iloc[k, :]['ville']
    equipement_test = data_cities['equipement'] == data_cities.iloc[k, :]['equipement']
    
    ca_py = data_cities[ equipement_test & ville_test & date_test ]['CA'].values
    
    if ca_py :
        CA_previous_year.append(ca_py[0])
    else :
        CA_previous_year.append(None)


data_cities['CA_previous_year'] = CA_previous_year

In [ ]:
CA_previous_year_same_day = []

for k in range(len(data_cities)) :
    date_test = data_cities['date'] == data_cities.iloc[k, :]['processed_previous_year_same_day']
    ville_test = data_cities['ville'] == data_cities.iloc[k, :]['ville']
    equipement_test = data_cities['equipement'] == data_cities.iloc[k, :]['equipement']
    
    ca_py = data_cities[ equipement_test & ville_test & date_test ]['CA'].values
    
    if ca_py :
        CA_previous_year_same_day.append(ca_py[0])
    else :
        CA_previous_year_same_day.append(None)


data_cities['CA_previous_year_same_day'] = CA_previous_year_same_day

In [ ]:
data_cities['weekday'] = data_cities['processed_date'].apply(lambda row : calendar.day_name[row.weekday()] )

In [ ]:
data_cities['is_weekend'] = data_cities['weekday'].apply(lambda row : row in ['Sunday', 'Saturday'] )

In [ ]:
years_list = data_cities.loc[:, 'date'].apply(lambda row : int(row.split('-', 2)[0])).unique()
#years_list

In [ ]:
hols = []
for date in holidays.France(years=years_list).items() :
    hols.append(date[0])
    
data_cities['is_bankholiday'] = data_cities['processed_date'].apply(lambda row : row in hols)

In [ ]:
from datetime import date
d = SchoolHolidayDates()
data_cities['is_school_holiday_A'] = data_cities['processed_date'].apply(lambda row : d.is_holiday_for_zone(date(row.year, row.month, row.day), 'A') )
data_cities['is_school_holiday_B'] = data_cities['processed_date'].apply(lambda row : d.is_holiday_for_zone(date(row.year, row.month, row.day), 'B') )
data_cities['is_school_holiday_C'] = data_cities['processed_date'].apply(lambda row : d.is_holiday_for_zone(date(row.year, row.month, row.day), 'C') )

In [ ]:
data_cities['distance_between_closest_bank_holiday'] = \
data_cities['processed_date'].apply(lambda row : abs(min(np.array(hols) - date(row.year, row.month, row.day))) )

In [ ]:
data_cities['distance_between_closest_bank_holiday'] 

In [ ]:
data_cities[(data_cities['year'] == 2017) & (data_cities['month'] == 1)]['CA'].sum()

In [ ]:
data_cities[['year', 'CA']]

In [ ]:
min(np.array(hols) - date(2017, 1, 1))

In [ ]:
data_cities['year'] = [d.year for d in data_cities['processed_date']]
data_cities['month'] = [d.month for d in data_cities['processed_date']]
data_cities['time'] = data_cities['processed_date'].apply(lambda row : str(row.year) + '-' + str(row.month).zfill(2) )
CA_plot = data_cities[['month', 'time', 'CA']].groupby(by=['time']).sum().sort_values(by='time').reset_index()
CA_plot_equipement = data_cities[['time', 'CA', 'equipement']].groupby(by=['time', 'equipement']).sum().sort_values(by='time').reset_index()



In [ ]:
import matplotlib.pyplot as plt

def plot_df(df, x, y, title='', xlabel='Temps mensuel', ylabel='CA', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.xticks(rotation='vertical')
    plt.plot(x, y, color='tab:blue')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

In [ ]:
plot_df(CA_plot, x=CA_plot['time'], y=CA_plot['CA'], title='Evolution du CA mensuel de 2017 à 2019') 

In [ ]:
CA_plot_ordinateur = CA_plot_equipement[CA_plot_equipement['equipement'] == 'ordinateur']
CA_plot_telephone = CA_plot_equipement[CA_plot_equipement['equipement'] == 'telephone']

In [ ]:
plot_df(CA_plot_ordinateur, x=CA_plot_ordinateur['time'], y=CA_plot_ordinateur['CA'], title='Evolution du CA ordinateurs mensuel de 2017 à 2019')  

In [ ]:
plot_df(CA_plot_telephone, x=CA_plot_telephone['time'], y=CA_plot_telephone['CA'], title='Evolution du CA telephone mensuel de 2017 à 2019')  

In [ ]:
#data_cities = data_cities[['date', 'equipement', 'ville', 'CA']].sort_values(by=['date'], ascending=False)

In [ ]:
data_cities.drop(columns=['processed_date', 'processed_previous_year_same_day', 'previous_year_same_day', 'previous_year', 'processed_previous_year', 'year', 'month', 'time'], inplace=True)
data_cities.to_csv('processed_data_v1', index=False)

In [ ]:
tst = pd.read_csv('processed_data_v1')
tst